In [2]:
import pandas as pd
import time
import geopandas
from glob import glob
from shapely.ops import cascaded_union
import pickle

In [3]:
bids = pd.read_csv("data/all_bids.csv")[["round", "bidder", "tier", "latency", "census_id"]]
bids["state"] = bids["census_id"]//10000000000
bids

,round,bidder,tier,latency,census_id,state
0,1,Rainbow Communications LLC,Gigabit,Low,200050816001,20
1,1,Rainbow Communications LLC,Gigabit,Low,200050816002,20
2,1,Rainbow Communications LLC,Gigabit,Low,200050816003,20
3,1,Rainbow Communications LLC,Gigabit,Low,200050816005,20
4,1,Rainbow Communications LLC,Gigabit,Low,200050817002,20
...,...,...,...,...,...,...
4331242,19,NexTier Consortium,Gigabit,Low,484690014001,48
4331243,19,NexTier Consortium,Gigabit,Low,484690015011,48
4331244,19,NexTier Consortium,Gigabit,Low,484690015012,48
4331245,19,Segnem Egere Consortium,Gigabit,Low,50850208001,5


In [4]:
existing = pd.read_csv("data/existing_service.csv", encoding="latin")
existing["bg"] = existing["BlockCode"]//1000
existing  = existing.loc[(existing["MaxAdDown"]>=25) & (existing["MaxAdUp"]>=3) & (existing["TechCode"]!=60)]
existing["state"] = existing["bg"]//10000000000
existing

,LogRecNo,Provider_Id,FRN,ProviderName,DBAName,HoldingCompanyName,HocoNum,HocoFinal,StateAbbr,BlockCode,TechCode,Consumer,MaxAdDown,MaxAdUp,Business,bg,state
0,1,59590,17678236,Webformix Company,Webformix,Webformix Company,290118,Webformix Company,OR,410139501002019,70,1,40.0,10.0,1,410139501002,41
1,2,59590,17678236,Webformix Company,Webformix,Webformix Company,290118,Webformix Company,OR,410139503002023,70,1,40.0,10.0,1,410139503002,41
2,3,59590,17678236,Webformix Company,Webformix,Webformix Company,290118,Webformix Company,OR,410139502002027,70,1,40.0,10.0,1,410139502002,41
3,4,59590,17678236,Webformix Company,Webformix,Webformix Company,290118,Webformix Company,OR,410139501003004,70,1,40.0,10.0,1,410139501003,41
4,5,59590,17678236,Webformix Company,Webformix,Webformix Company,290118,Webformix Company,OR,410139504001109,70,1,40.0,10.0,1,410139504001,41
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
42151570,42151571,79963,21999305,OSNET Wireless Corp.,OSNET Wireless,OSNET Wireless Corp.,420091,Osnet Wireless Corp.,PR,720054006003006,70,1,30.0,10.0,1,720054006003,72
42151571,42151572,79963,21999305,OSNET Wireless Corp.,OSNET Wireless,OSNET Wireless Corp.,420091,Osnet Wireless Corp.,PR,720054006003005,70,1,30.0,10.0,1,720054006003,72
42151572,42151573,79963,21999305,OSNET Wireless Corp.,OSNET Wireless,OSNET Wireless Corp.,420091,Osnet Wireless Corp.,PR,720054006003004,70,1,30.0,10.0,1,720054006003,72
42151573,42151574,79963,21999305,OSNET Wireless Corp.,OSNET Wireless,OSNET Wireless Corp.,420091,Osnet Wireless Corp.,PR,720054006003003,70,1,30.0,10.0,1,720054006003,72


In [5]:
existing_bg = set(existing["bg"].tolist())
bid_bg = set(bids["census_id"].tolist())
print(len(existing_bg))
print(len(bid_bg))
print(len(existing_bg.intersection(bid_bg)))

219391
61766
61021


In [6]:
eligible = pd.read_csv("data/auction904_updated_block_cbg.csv")
eligible_bg = eligible["cbg_id"].tolist()
print(len(set(eligible_bg)))
eligible_b = eligible["block_id"].tolist()
existing_b = set(existing["BlockCode"].tolist())
print(existing.loc[existing["BlockCode"] == 170679543005184])
print(len(existing_b))
print(len(eligible_b))
intersect = existing_b.intersection(eligible_b)
print(len(existing_b.intersection(eligible_b)))
print(list(intersect)[:5])


61766
        LogRecNo  Provider_Id      FRN                         ProviderName  \
307522    307523        59866  3576352  Frontier Communications Corporation   

                                    DBAName  \
307522  Frontier Communications Corporation   

                         HoldingCompanyName  HocoNum  \
307522  Frontier Communications Corporation   130258   

                                  HocoFinal StateAbbr        BlockCode  \
307522  Frontier Communications Corporation        IL  170679543005184   

        TechCode  Consumer  MaxAdDown  MaxAdUp  Business            bg  state  
307522        12         1      115.0      7.0         1  170679543005     17  
8063170
786952
292114
[170679543005184, 170679543005185, 170679543005186, 170679543005187, 170679543005188]


In [7]:
shapefiles = glob("data/all_data/*.shp")
county_shapes = []
for file in  shapefiles:
    county_shapes.append(geopandas.read_file(file))

county_shapes = pd.concat(county_shapes)[["STATEFP20", "COUNTYFP20", "TRACTCE20","GEOID20", "geometry"]]
county_shapes["GEOID20"] = pd.to_numeric(county_shapes["GEOID20"])
county_shapes

,STATEFP20,COUNTYFP20,TRACTCE20,GEOID20,geometry
0,12,086,008415,120860084151,"POLYGON ((-80.36757 25.66494, -80.36491 25.665..."
1,12,086,008415,120860084152,"POLYGON ((-80.35607 25.66529, -80.35605 25.665..."
2,12,086,008415,120860084153,"POLYGON ((-80.36721 25.65017, -80.36511 25.650..."
3,12,086,016800,120860168002,"POLYGON ((-80.42949 25.71012, -80.42940 25.710..."
4,12,086,016800,120860168001,"POLYGON ((-80.43107 25.71388, -80.43044 25.713..."
...,...,...,...,...,...
3920,01,079,979501,10799795012,"POLYGON ((-87.20741 34.55710, -87.20740 34.557..."
3921,01,079,979502,10799795023,"POLYGON ((-87.24162 34.53591, -87.24161 34.536..."
3922,01,079,979201,10799792011,"POLYGON ((-87.43611 34.72743, -87.43610 34.727..."
3923,01,079,979202,10799792022,"POLYGON ((-87.45696 34.61352, -87.45693 34.613..."


In [8]:
state_shapes = county_shapes.groupby("STATEFP20")["geometry"].apply(list).reset_index(name="geometry")
state_shapes["geometry"] = state_shapes["geometry"].apply(cascaded_union)
state_shapes = geopandas.GeoDataFrame(state_shapes)
for index, row in state_shapes.iterrows():
    neighbors = state_shapes[~state_shapes.geometry.disjoint(row['geometry'])]
    state_shapes.at[index, "neighbors"] = ", ".join(neighbors.STATEFP20.tolist())
state_shapes

,STATEFP20,geometry,neighbors
0,01,"POLYGON ((-85.24009 31.00088, -85.24204 31.000...","01, 12, 13, 28, 47"
1,02,"MULTIPOLYGON (((-179.23023 51.28393, -179.2256...",02
2,04,"POLYGON ((-109.49449 31.33398, -109.49874 31.3...","04, 06, 08, 32, 35, 49"
3,05,"POLYGON ((-91.19596 33.13342, -91.19609 33.133...","05, 22, 28, 29, 40, 47, 48"
4,06,"MULTIPOLYGON (((-119.63607 33.28071, -119.6347...","04, 06, 32, 41"
5,08,"POLYGON ((-104.43399 36.99374, -104.43521 36.9...","04, 08, 20, 31, 35, 40, 49, 56"
6,09,"POLYGON ((-73.64983 40.97920, -73.65637 40.984...","09, 25, 36, 44"
7,10,"POLYGON ((-75.42864 38.45277, -75.43031 38.452...","10, 24, 34, 42"
8,11,"POLYGON ((-77.03919 38.80050, -77.03913 38.800...","11, 24, 51"
9,12,"MULTIPOLYGON (((-83.10874 24.62949, -83.10711 ...","01, 12, 13"


In [ ]:
with open("neighbors.log", "w") as log:
    total_time = 0
    processed_bid_dict = {}
    for state_code in state_shapes["STATEFP20"].tolist():
        start = time.time()

        state_bids = bids.loc[bids["state"]==int(state_code)]
        state_bids = state_bids.groupby("census_id")["bidder"].apply(set).reset_index(name="bidder")
        state_bids = state_bids.merge(county_shapes, left_on="census_id", right_on="GEOID20")

        neighbors = state_shapes.loc[state_shapes["STATEFP20"] == state_code]["neighbors"].item().split(", ")
        neighbors = [int(n) for n in neighbors]
        state_existing = existing.loc[existing["state"].isin(neighbors)]
        state_existing = state_existing.groupby("bg")["DBAName"].apply(set).reset_index(name="DBAName")
        state_existing = state_existing.merge(county_shapes, left_on="bg", right_on="GEOID20")
        state_existing = geopandas.GeoDataFrame(state_existing)

        for index, row in state_bids.iterrows():
            neighbors = state_existing[~state_existing.geometry.disjoint(row['geometry'])]
            ids = neighbors.GEOID20.tolist()
            names = neighbors.DBAName.tolist()
            out = {}
            for idx, name_list in zip(ids, names):
                for name in name_list:
                    running = out.get(name, [])
                    running.append(idx)
                    out[name] = running
            state_bids.at[index, "neighbors"] = [out]
        state_time = time.time()-start
        log.write(f"{state_code} {state_time}\n")
        total_time += state_time
        processed_bid_dict[state_code] = state_bids

    log.write("final time", time.time()-start)


In [ ]:
with open('neighbors.pkl', 'wb') as f:
    pickle.dump(f, processed_bid_dict)